# First neural network fitting for THz-TDS material parameter extraction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import Callback
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from IPython.display import display, clear_output

## Prepare the data

In [2]:
# Prepare NN training data
df = pd.read_csv('training_data/NN_data_2025-01-24_12-39-28.csv') # Replace this with a generated dataset as these are too large to store on github.
print('df head:')
display(df.head())

df.describe()

FileNotFoundError: [Errno 2] No such file or directory: 'training_data/NN_data_2025-01-24_12-39-28.csv'

In [ ]:
# Split up data sets
X = df[['H', 'phi', 'freq']].head(int(len(df['freq'])))
Y = df[['n', 'k']].head(int(len(df['freq'])))

# Normalize frequencies to prevent weighting issues
freq_scaler = MinMaxScaler(feature_range=(0.01, 1))

X[['freq']] = freq_scaler.fit_transform(X[['freq']])

# Prepare validation datasets for plotting
N = 4   # Number of included datasets

X_val = X.head(N*64)
Y_val = Y.head(N*64)


# Check DataFrames
display(X)
display(Y)

# Check value bounds of results
print('Target bounds:')
print(f'n: min: {min(Y["n"])}, max: {max(Y["n"])}')
print(f'k: min: {min(Y["k"])}, max: {max(Y["k"])}')

In [ ]:
display(X_val)
display(Y_val)

In [ ]:
plt.plot(X['freq'].head(64),X['H'].head(64))

In [ ]:
# Split into training and testing datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Output to check shapes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")

## First model (basic)

In [8]:
# Define the model

# TODO: test deeper network. 5x16? 

def build_model_basic(input_shape, output_shape, hidden_layers: int = 1):
    # Input layer
    inputs = Input(shape=input_shape, name="Input_Layer")
    
    # Use loops to add main hidden layers
    for i in range(hidden_layers):
        x = Dense(256, activation='tanh')(inputs) 
        #x = BatchNormalization()(x)
        #x = Dropout(0.3)(x)

    # Output layer
    outputs = Dense(output_shape, activation='linear', name="Output_Layer")(x)

    # Build the model
    model = Model(inputs=inputs, outputs=outputs, name="model_basic")
    return model

In [ ]:
# Model parameters
input_shape = (3,)  # Number of features in X
output_shape = 2    # Number of outputs in Y

# Build the model
model_basic = build_model_basic(input_shape, output_shape)

# Compile the model
model_basic.compile(optimizer='adam', 
                    loss='mse',  # Mean Squared Error for regression
                    metrics=['mae'])  # Mean Absolute Error as an additional metric

# Summary
model_basic.summary()

In [ ]:
# Train the model
history = model_basic.fit(X_train, Y_train,
                          validation_data=(X_test, Y_test),
                          epochs=10,
                          batch_size=128,
                          verbose=1)

# Evaluate the model
results = model_basic.evaluate(X_test, Y_test, verbose=0)
print(f"Test Loss (MSE): {results[0]:.4f}")
print(f"Test MAE: {results[1]:.4f}")


In [ ]:
# Plot training and validation metrics
def plot_training_history(history):
    # Extract metrics
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    mae = history.history['mae']
    val_mae = history.history['val_mae']
    epochs = range(1, len(loss) + 1)
    
    # Plot Loss
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'b-', label='Training Loss')
    plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.grid(True)
    
    # Plot MAE
    plt.subplot(1, 2, 2)
    plt.plot(epochs, mae, 'b-', label='Training MAE')
    plt.plot(epochs, val_mae, 'r-', label='Validation MAE')
    plt.title('Training and Validation MAE')
    plt.xlabel('Epochs')
    plt.ylabel('Mean Absolute Error (MAE)')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Call the function to plot the metrics
plot_training_history(history)

In [ ]:
test = model_basic.predict(X_val.head(64))[:,0]

test

In [ ]:
# Reshape the data: Y_val has 4 sets, each with 64 points
n_data = Y_val['n'].values.reshape(4, 64)  # Reshape to 4 sets of 64 points each
k_data = Y_val['k'].values.reshape(4, 64)

freqs = X_val['freq'].values.reshape(4, 64)
# freqs = (freqs * 1e-12) / (2 * np.pi)

# Make predictions using the model
predictions = model_basic.predict(X_val)
predictions_df = pd.DataFrame(predictions, columns=['n', 'k'])

# Reshape the data: predicted data has 4 sets, each with 64 points
n_pred = predictions_df['n'].values.reshape(4, 64)  # Reshape to 4 sets of 64 points each
k_pred = predictions_df['k'].values.reshape(4, 64)

In [ ]:
# Create a 4x2 grid of subplots
fig, axes = plt.subplots(2, 4, figsize=(16, 8))  # 4 wide, 2 tall grid

# Plotting the n-related data (top row)
for i in range(4):  # There are 4 columns in the top row
    ax = axes[0, i]  # Access the subplot in the top row
    ax.plot(freqs[i], n_data[i], 'b-', label='n val')  # Blue for actual n
    ax.plot(freqs[i], n_pred[i], 'g-', label='n pred')  # Green for predicted n
    ax.set_title(f'n Plot {i + 1}')
    ax.set_xlabel('Normalized Frequency')
    ax.set_ylabel('n')
    ax.legend()

# Plotting the k-related data (bottom row)
for i in range(4):  # There are 4 columns in the bottom row
    ax = axes[1, i]  # Access the subplot in the bottom row
    ax.plot(freqs[i], k_data[i], 'r-', label='k val')  # Red for actual k
    ax.plot(freqs[i], k_pred[i], 'orange', label='k pred')  # Orange for predicted k
    ax.set_title(f'k Plot {i + 1}')
    ax.set_xlabel('Normalized Frequency')
    ax.set_ylabel('k')
    ax.legend()

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

### Custom live plotting of training

In [ ]:
'''
# Define the custom callback
class LivePlotCallback(Callback):
    def __init__(self):
        super(LivePlotCallback, self).__init__()
        self.epochs = []
        self.loss = []
        self.val_loss = []
        self.mae = []
        self.val_mae = []

    def on_epoch_end(self, epoch, logs=None):
        # Append metrics for each epoch
        self.epochs.append(epoch + 1)
        self.loss.append(logs['loss'])
        self.val_loss.append(logs['val_loss'])
        self.mae.append(logs['mae'])
        self.val_mae.append(logs['val_mae'])

        # Clear output and plot
        clear_output(wait=True)
        plt.figure(figsize=(14, 6))

        # Plot Loss
        plt.subplot(1, 2, 1)
        plt.plot(self.epochs, self.loss, 'b-', label='Training Loss')
        plt.plot(self.epochs, self.val_loss, 'r-', label='Validation Loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss (MSE)')
        plt.legend()
        plt.grid(True)

        # Plot MAE
        plt.subplot(1, 2, 2)
        plt.plot(self.epochs, self.mae, 'b-', label='Training MAE')
        plt.plot(self.epochs, self.val_mae, 'r-', label='Validation MAE')
        plt.title('Training and Validation MAE')
        plt.xlabel('Epochs')
        plt.ylabel('Mean Absolute Error (MAE)')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.show()

# Train the model with the custom callback
history = model_basic.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=50,
    batch_size=256,
    callbacks=[LivePlotCallback()],  # Add custom callback
    verbose=1
)
'''

## More complex models

### Use scaling on the same model to make mae metric less misleading

In [ ]:
# Initialize scalers
scaler_n = MinMaxScaler(feature_range=(0, 1))
scaler_k = MinMaxScaler(feature_range=(0, 1))

# Normalize outputs
Y_train_scaled = Y_train.copy()
Y_test_scaled = Y_test.copy()

# Scale the columns explicitly
Y_train_scaled['n'] = scaler_n.fit_transform(Y_train[['n']])
Y_train_scaled['k'] = scaler_k.fit_transform(Y_train[['k']])
Y_test_scaled['n'] = scaler_n.transform(Y_test[['n']])
Y_test_scaled['k'] = scaler_k.transform(Y_test[['k']])

# Build the model for scaled data
model_basic_scaled = build_model_basic(input_shape, output_shape)

# Compile the model
model_basic_scaled.compile(optimizer='adam', 
                    loss='mse',  # Mean Squared Error for regression
                    metrics=['mae'])  # Mean Absolute Error as an additional metric

# Summary
model_basic_scaled.summary()

# Train the model on scaled outputs
history_scaled = model_basic_scaled.fit(
    X_train, Y_train_scaled,
    validation_data=(X_test, Y_test_scaled),
    epochs=10,
    batch_size=256,
    verbose=1
)

# Rescale predictions for evaluation
predictions_scaled = model_basic_scaled.predict(X_test)

# Convert predictions to a DataFrame for easier handling
predictions_scaled_df = pd.DataFrame(predictions_scaled, columns=['n', 'k'])

# Rescale each output
predictions_scaled_df['n'] = scaler_n.inverse_transform(predictions_scaled_df[['n']])
predictions_scaled_df['k'] = scaler_k.inverse_transform(predictions_scaled_df[['k']])

In [ ]:
plot_training_history(history_scaled)

In [ ]:
# Create predictions for the validation set
val_predicted = model_basic_scaled.predict(X_val)

# Convert predictions to a DataFrame for easier handling
val_predicted_scaled_df = pd.DataFrame(val_predicted, columns=['n', 'k'])

# Rescale each output
val_predicted_scaled_df['n'] = scaler_n.inverse_transform(val_predicted_scaled_df[['n']])
val_predicted_scaled_df['k'] = scaler_k.inverse_transform(val_predicted_scaled_df[['k']])

# Reshape the data: predicted data has 4 sets, each with 64 points
n_pred_data = val_predicted_scaled_df['n'].values.reshape(4, 64)  # Reshape to 4 sets of 64 points each
k_pred_data = val_predicted_scaled_df['k'].values.reshape(4, 64)

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 8))  # 4 wide, 2 tall grid

# Plotting the n-related data
for i in range(4):  
    ax = axes[0, i]  
    ax.plot(freqs[i], n_data[i], 'b-', label='n val') 
    ax.plot(freqs[i], n_pred_data[i], 'g-', label='n pred')  
    ax.set_title(f'n Plot {i + 1}: Scaled Training')
    ax.set_xlabel('Frequency')
    ax.set_ylabel('n')
    ax.legend()

# Plotting the k-related data
for i in range(4):  
    ax = axes[1, i]  
    ax.plot(freqs[i], k_data[i], 'r-', label='k val')  
    ax.plot(freqs[i], k_pred_data[i], 'orange', label='k pred')  
    ax.set_title(f'k Plot {i + 1}: Scaled Training')
    ax.set_xlabel('Frequency')
    ax.set_ylabel('k')
    ax.legend()


plt.tight_layout()

plt.show()

## Add noise to signals to see if it improves fitting

In [20]:
# Define the standard deviation of the Gaussian noise
std_dev_H = 0.01  
std_dev_phi = 1 

# Add Gaussian noise to each column
X_noisy = X.copy()
X_noisy['H'] += np.random.normal(0, std_dev_H, size=X['H'].shape)
X_noisy['phi'] += np.random.normal(0, std_dev_phi, size=X['phi'].shape)

In [ ]:
plt.plot(X_noisy['freq'].head(64), X_noisy['H'].head(64))

In [22]:
X_train_noisy, X_test_noisy, Y_train_noisy, Y_test_noisy = train_test_split(X_noisy, Y, test_size=0.3, random_state=42)

In [ ]:
display(X_train_noisy.head())
display(Y_train_noisy.head())

In [ ]:
# Initialize scalers for noisy X features
noisy_scaler_n = MinMaxScaler(feature_range=(0, 1))
noisy_scaler_k = MinMaxScaler(feature_range=(0, 1))

# Normalize outputs
Y_train_scaled_noisy = Y_train_noisy.copy()
Y_test_scaled_noisy = Y_test_noisy.copy()

# Scale the columns explicitly
Y_train_scaled_noisy['n'] = noisy_scaler_n.fit_transform(Y_train_noisy[['n']])
Y_train_scaled_noisy['k'] = noisy_scaler_k.fit_transform(Y_train_noisy[['k']])
Y_test_scaled_noisy['n'] = noisy_scaler_n.transform(Y_test_noisy[['n']])
Y_test_scaled_noisy['k'] = noisy_scaler_k.transform(Y_test_noisy[['k']])

# Build the model for scaled data
model_basic_noisy_scaled = build_model_basic(input_shape, output_shape)

# Compile the model
model_basic_noisy_scaled.compile(optimizer='adam', 
                    loss='mse',  # Mean Squared Error for regression
                    metrics=['mae'])  # Mean Absolute Error as an additional metric

# Summary
model_basic_noisy_scaled.summary()

# Train the model on scaled outputs
history_noisy_scaled = model_basic_noisy_scaled.fit(
    X_train_noisy, Y_train_scaled_noisy,
    validation_data=(X_test_noisy, Y_test_scaled_noisy),
    epochs=10,
    batch_size=256,
    verbose=1
)

In [ ]:
plot_training_history(history_noisy_scaled)

In [ ]:
noisy_predictions = model_basic_noisy_scaled.predict(X_val)

noisy_predictions_df = pd.DataFrame(noisy_predictions, columns=['n_scaled', 'k_scaled'])

# Rescale each output
noisy_predictions_df['n'] = noisy_scaler_n.inverse_transform(noisy_predictions_df[['n_scaled']])
noisy_predictions_df['k'] = noisy_scaler_k.inverse_transform(noisy_predictions_df[['k_scaled']])

# Reshape the data: predicted data has 4 sets, each with 64 points
n_pred_data_noisy = noisy_predictions_df['n'].values.reshape(4, 64)  # Reshape to 4 sets of 64 points each
k_pred_data_noisy = noisy_predictions_df['k'].values.reshape(4, 64)

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 8))  # 4 wide, 2 tall grid

# Plotting the n-related data
for i in range(4):  
    ax = axes[0, i]  
    ax.plot(freqs[i], n_data[i], 'b-', label='n val') 
    ax.plot(freqs[i], n_pred_data_noisy[i], 'g-', label='n pred')  
    ax.set_title(f'n Plot {i + 1}: Scaled Training')
    ax.set_xlabel('Frequency')
    ax.set_ylabel('n')
    ax.legend()

# Plotting the k-related data
for i in range(4):  
    ax = axes[1, i]  
    ax.plot(freqs[i], k_data[i], 'r-', label='k val')  
    ax.plot(freqs[i], k_pred_data_noisy[i], 'orange', label='k pred')  
    ax.set_title(f'k Plot {i + 1}: Scaled Training')
    ax.set_xlabel('Frequency')
    ax.set_ylabel('k')
    ax.legend()


plt.tight_layout()

plt.show()